#import all required libraries

In [28]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [29]:
PINECONE_API_KEY = "c9262956-eeb0-4fcb-a75a-a47fe49e42fe"
PINECONE_API_ENV = "us-west-2"

In [30]:
#Extract data from the PDF
def load_pdf(data):
    loader = PyPDFLoader(data)

    
    documents = loader.load()

    return documents
extracted_data = load_pdf("C:/Users/Sahana/End-to-end-Medical-Chatbot-Using-Llama2/data/Medical_book.pdf")


# extracted_data

In [31]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [32]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 7020



# text_chunks

In [33]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [34]:
embeddings = download_hugging_face_embeddings()

In [35]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [36]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))


Length 384


In [37]:
PINECONE_API_KEY = "c9262956-eeb0-4fcb-a75a-a47fe49e42fe"
PINECONE_API_ENV = "us-west-2"

In [38]:
from pinecone import Pinecone

# Initialize the Pinecone client
pc= Pinecone(api_key="c9262956-eeb0-4fcb-a75a-a47fe49e42fe", environment="us-west-2")




In [39]:
import pinecone
from langchain.vectorstores import Pinecone as LangchainPinecone

index_name="medical-chatbot"



In [45]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Pinecone as LangchainPinecone
from pinecone import Pinecone

# Set Pinecone API key and environment
os.environ["PINECONE_API_KEY"] = "c9262956-eeb0-4fcb-a75a-a47fe49e42fe"
os.environ["PINECONE_ENVIRONMENT"] = "us-west-2"

# Extract data from the PDF
def load_pdf(data):
    loader = PyPDFLoader(data)
    documents = loader.load()
    return documents

# Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

# Download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

# Main execution
if __name__ == "__main__":
    # Load and process the PDF
    extracted_data = load_pdf("C:/Users/Sahana/End-to-end-Medical-Chatbot-Using-Llama2/data/Medical_book.pdf")
    text_chunks = text_split(extracted_data)
    print("Length of my chunks:", len(text_chunks))

    # Initialize embeddings
    embeddings = download_hugging_face_embeddings()

    # Initialize Pinecone
    pc = Pinecone()
    index_name = "medical-chatbot"

    # Initialize LangchainPinecone
    docsearch = LangchainPinecone.from_existing_index(index_name=index_name, embedding=embeddings)

    # Add text chunks to Pinecone
    for i, chunk in enumerate(text_chunks):
        docsearch.add_texts([chunk.page_content], metadatas=[{"source": f"chunk_{i}"}])

    print("All chunks have been added to Pinecone index.")

Length of my chunks: 7020


KeyboardInterrupt: 

##I lost my network when storing emebeding so it was overriding the data so I have stopped 

In [50]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [51]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [57]:
# Initialize the language model
model_path = os.path.abspath("C:/Users/Sahana/End-to-end-Medical-Chatbot-Using-Llama2/model/llama-2-7b-chat.ggmlv3.q4_0.bin")

llm = CTransformers(
        model=model_path,
        model_type="llama",
        config={'max_new_tokens': 512, 'temperature': 0.8}
    )

In [59]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

#HI guys I recommend using Gpu to run llama2 I'm interrrupting the code because my laptop is using CPU

In [60]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

C:\Users\Sahana\AppData\Local\Temp\ipykernel_11732\4216317267.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result=qa({"query": user_input})


KeyboardInterrupt: 